In [18]:
import json

with open('review_list_그랜드 하얏트 제주.json','r',encoding='UTF8') as f:
    review_list = json.load(f)

In [20]:
good_cnt, bad_cnt = 0, 0

for review in review_list:
    if review['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

In [21]:
reviews_good, reviews_bad = [], []

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

In [22]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='UTF8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews("review_list_그랜드 하얏트 제주.json")

In [11]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 3문장 내로 요약해줘:"""

In [23]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

## Zero-shot

In [30]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt).choices[0].message.content}")

좋은 리뷰 요약: 전망이 좋고 뷔페식당도 좋았으나 주변 상점은 조금 약한 편으로 보였습니다. 호캉스를 즐기기에 좋은 호텔로 후회 없는 선택이었고, 수영장과 38층 포차에서의 썬셋은 특히 좋았습니다. 부모님과 함께하기에 좋은 호텔이며, 깨끗하고 넓은 공간과 최신 시설로 만족스러운 숙박을 경험할 수 있었습니다.룸에서 공항뷰를 즐기며 휴식을 취할 수 있는 호텔로, 서비스와 룸의 만족도가 높았습니다.
나쁜 리뷰 요약: 이 숙소는 객실과 침구가 마음에 들었지만, 대부분의 고객이 중국인이라는 점이 냄새로 불편함을 느끼게 합니다. 조식은 별로였고, 제주스러운 분위기가 부족해서 아쉬운 점이 있습니다. 다만, 객실은 호텔 스위트 수준으로 만족스러웠고, 조식과 수영장은 좋았지만 직원의 친절도가 차이가 있어 아쉬운 점이 있습니다. 또한, 호텔은 깨끗하고 깔끔하며 고층에 위치해 좋았지만, 한국인 투숙객이 없어서 조금 외로웠고, 중국인이 많아서 불편함을 느낄 수 있습니다. 뷰와 서비스는 좋았지만, 떡볶이 맛이 별로였고, 화재 발생 시 안내가 부족해 당황스러웠으며, 클럽 이용에도 불편함을 느낄 수 있었습니다. 전망과 시설은 좋지만, 가격 대비 만족도가 낮고, 중국인이 많아서 조식 포함 여부가 명확히 표시되어 있지 않아 아쉬운 점이 있습니다.


## One-Shot

In [38]:
summary_good = summarize(good, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content
summary_bad = summarize(bad, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content

In [34]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.

다음은 요약 예시입니다.
예시 요약 결과:
{summary_good}

아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_1shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_1shot).choices[0].message.content}")

좋은 리뷰 요약: 전망이 좋고 뷔페식당도 좋았습니다. 다만, 주변 상점은 조금 약한 편이었습니다. 자전거 국토종주 후 호캉스로 1박했는데 후회 없는 선택이었습니다. 수영장과 다른 시설은 좋았지만, 수영장에서 음식과 맥주를 즐길 수 없어서 조금 아쉬웠습니다. 부모님과 함께하기에 좋은 숙소로, 공항과 주변 맛집이 가까워 편리했습니다. 호텔하얏트제주는 깨끗하고 넓은 공간에 최신 시설과 친절한 서비스로 만족스러웠습니다.
나쁜 리뷰 요약: 숙소는 객실과 침구가 마음에 들었지만, 중국인 손님이 많아 냄새가 조금 불편했다는 의견이 있습니다. 조식은 별로였고 주변 분위기가 부족하다는 아쉬움도 표현되었습니다. 다른 리뷰에서는 객실 만족도가 높았지만 직원의 친절도에 차이가 있었다는 언급이 있습니다. 또한, 호텔 주변이 중국 관광객으로 붐비고 있어서 한국인 손님을 찾기 어려웠다는 점이 언급되었습니다. 호텔 자체는 깨끗하고 전망이 좋았지만, 가격 대비 만족도가 낮았고 중국인 손님이 많아서 조식 포함 여부가 명확히 안내되어 있지 않아 아쉬움을 표현한 리뷰도 있습니다.


## Two-Shot

In [39]:
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.

다음은 요약 예시입니다.
좋은 리뷰 요약 결과:
{summary_good}
나쁜 리뷰 요약 결과:
{summary_bad}

아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_2shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_2shot).choices[0].message.content}")

좋은 리뷰 요약: 전망이 좋고 뷔페식당도 좋았습니다. 다만, 주변 상점이 조금 약한 편으로 보였습니다. 호캉스를 즐기기에 좋은 호텔이었고, 수영장과 다양한 시설을 이용하여 후회 없는 선택이었습니다. 수영장에서 음식과 맥주를 즐길 수 없어서 조금 아쉬웠지만, 38층 포차에서 썬셋을 보며 술을 마실 수 있는 좋은 경험을 했습니다. 부모님과 함께 갔을 때 편리한 위치와 깨끗한 침구, 저렴한 룸서비스로 만족스러운 여행을 즐겼습니다. 호텔하얏트제주는 깨끗하고 넓은 공간에 최신 시설과 친절한 서비스로 대만족을 주는 숙소로, 적극 추천합니다. 호텔의 규모와 서비스, 엘리베이터 구분 등이 만족스러웠고, 수영장과 조식을 중식으로 바꾸어 먹을 수 있는 점도 좋았습니다. 전반적으로 좋은 호텔이지만 중국인 손님이 많아서 소음 문제와 중국어 안내문이 많다는 점이 아쉬웠습니다.
나쁜 리뷰 요약: 전반적으로 객실과 침구 상태에 대한 만족도가 높은 평이 많습니다. 다만, 호텔 고객 대부분이 중국인이라 냄새와 소음 문제가 지적되었으며, 주변에 제주스러운 분위기가 부족하다는 의견도 있습니다. 또한, 조식 포함 여부에 대한 정보가 명확하지 않아 불편함을 느낀 리뷰도 있습니다. 전반적으로 호텔은 깨끗하고 전망이 좋으나 중국인 투숙객이 많아 소음과 언어적 불편함이 있을 수 있다는 점이 강조됩니다. 재방문 의사에 대한 언급은 다소 분분합니다.
